In [4]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix
from fuzzywuzzy import process
import pickle
import numpy as np

/home/ongraph/Desktop/movie_recommed/venv/lib/python3.8/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [5]:
movies=pd.read_csv('TMDB 10000 Movies Dataset.csv')

In [6]:
movies.head()

,id,original_language,original_title,overview,popularity,release_date,title,vote_average,vote_count
0,851644,ko,20세기 소녀,Yeon-du asks her best friend Bora to collect a...,170.540,2022-10-06,20th Century Girl,8.7,290
1,238,en,The Godfather,"Spanning the years 1945 to 1955, a chronicle o...",86.518,1972-03-14,The Godfather,8.7,16988
2,278,en,The Shawshank Redemption,Framed in the 1940s for the double murder of h...,84.681,1994-09-23,The Shawshank Redemption,8.7,22748
3,240,en,The Godfather Part II,In the continuing saga of the Corleone crime f...,44.957,1974-12-20,The Godfather Part II,8.6,10293
4,667257,es,Cosas imposibles,"Matilde is a woman who, after the death of her...",32.859,2021-06-17,Impossible Things,8.6,299


In [7]:
movies.describe()

,id,popularity,vote_average,vote_count
count,1.000000e+04,10000.000000,10000.000000,10000.000000
mean,1.688979e+05,23.186472,6.681610,1592.082600
std,2.212173e+05,112.504114,0.705462,2724.194868
min,3.000000e+00,0.600000,5.300000,200.000000
25%,1.016750e+04,10.221750,6.100000,319.000000
50%,3.162500e+04,14.033500,6.700000,599.000000
75%,3.266500e+05,21.241750,7.200000,1508.000000
max,1.024530e+06,9137.939000,8.700000,32679.000000


In [8]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 10000 non-null  int64  
 1   original_language  10000 non-null  object 
 2   original_title     10000 non-null  object 
 3   overview           9994 non-null   object 
 4   popularity         10000 non-null  float64
 5   release_date       10000 non-null  object 
 6   title              10000 non-null  object 
 7   vote_average       10000 non-null  float64
 8   vote_count         10000 non-null  int64  
dtypes: float64(2), int64(2), object(5)
memory usage: 703.2+ KB


In [9]:
movies.isnull().sum()

id                   0
original_language    0
original_title       0
overview             6
popularity           0
release_date         0
title                0
vote_average         0
vote_count           0
dtype: int64

In [10]:
movies.dropna(subset=['overview'], inplace=True)

In [11]:
movies.isnull().sum()

id                   0
original_language    0
original_title       0
overview             0
popularity           0
release_date         0
title                0
vote_average         0
vote_count           0
dtype: int64

In [12]:
movies.columns

Index(['id', 'original_language', 'original_title', 'overview', 'popularity',
       'release_date', 'title', 'vote_average', 'vote_count'],
      dtype='object')

In [13]:
movies=movies[['id','title','overview']]

In [14]:
movies

,id,title,overview
0,851644,20th Century Girl,Yeon-du asks her best friend Bora to collect a...
1,238,The Godfather,"Spanning the years 1945 to 1955, a chronicle o..."
2,278,The Shawshank Redemption,Framed in the 1940s for the double murder of h...
3,240,The Godfather Part II,In the continuing saga of the Corleone crime f...
4,667257,Impossible Things,"Matilde is a woman who, after the death of her..."
...,...,...,...
9995,11375,Hollywood Homicide,"Joe Gavilan and his new partner K. C. Calden, ..."
9996,298614,Do Not Disturb,"Michel, who's crazy about jazz, has just found..."
9997,50357,Apollo 18,"Officially, Apollo 17 was the last manned miss..."
9998,97795,Apartment 143,A team of parapsychologists sets out to invest...


In [15]:
movies['overview'].describe

<bound method NDFrame.describe of 0       Yeon-du asks her best friend Bora to collect a...
1       Spanning the years 1945 to 1955, a chronicle o...
2       Framed in the 1940s for the double murder of h...
3       In the continuing saga of the Corleone crime f...
4       Matilde is a woman who, after the death of her...
                              ...                        
9995    Joe Gavilan and his new partner K. C. Calden, ...
9996    Michel, who's crazy about jazz, has just found...
9997    Officially, Apollo 17 was the last manned miss...
9998    A team of parapsychologists sets out to invest...
9999    A street sweeper who cleans up after grisly ac...
Name: overview, Length: 9994, dtype: object>

In [16]:
from sklearn.feature_extraction.text import CountVectorizer

In [17]:
cv=CountVectorizer(max_features=10000,stop_words='english')

In [18]:
cv

CountVectorizer(max_features=10000, stop_words='english')

In [19]:
vector=cv.fit_transform(movies['overview']).toarray()

In [20]:
print(vector[4])

[0 0 0 ... 0 0 0]


In [21]:
vector.shape

(9994, 10000)

In [22]:
from sklearn.metrics.pairwise import cosine_similarity

In [23]:
similarity=cosine_similarity(vector)

In [24]:
similarity

array([[1.        , 0.        , 0.0244998 , ..., 0.02178551, 0.        ,
        0.        ],
       [0.        , 1.        , 0.0285831 , ..., 0.02541643, 0.        ,
        0.        ],
       [0.0244998 , 0.0285831 , 1.        , ..., 0.        , 0.        ,
        0.08318903],
       ...,
       [0.02178551, 0.02541643, 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.08318903, ..., 0.        , 0.        ,
        1.        ]])

In [25]:
movies[movies['title']=='The Godfather'].index[0]

1

In [26]:
distance=sorted(list(enumerate(similarity[1])),reverse=True,key=lambda vector:vector[1])

In [27]:
print(len(distance))

9994


In [28]:
for i in distance[0:5]:
    print(movies.iloc[i[0]].title)

The Godfather
The Godfather Part II
Kind Hearts and Coronets
Still Walking
On My Skin


In [29]:
def recommendation(film):
    index=movies[movies['title']==film].index[0]
    distance=sorted(list(enumerate(similarity[index])),reverse=True,key=lambda vector:vector[1])
    for i in distance[0:5]:
        print(movies.iloc[i[0]].title)

In [30]:
recommendation('Iron Man')

Iron Man
Clown
Iron Man 2
The Wizards Return: Alex vs. Alex
G.O.R.A.


In [31]:
import joblib

In [32]:
joblib.dump(movies,'movie_list.pkl')

['movie_list.pkl']

In [38]:
joblib.dump(movies,'movie_list.joblib')

['movie_list.joblib']

In [33]:
joblib.dump(similarity,'similarity.pkl')

['similarity.pkl']

In [39]:
joblib.dump(similarity,'similarity.joblib')

['similarity.joblib']

In [34]:
joblib.load('movie_list.pkl')

,id,title,overview
0,851644,20th Century Girl,Yeon-du asks her best friend Bora to collect a...
1,238,The Godfather,"Spanning the years 1945 to 1955, a chronicle o..."
2,278,The Shawshank Redemption,Framed in the 1940s for the double murder of h...
3,240,The Godfather Part II,In the continuing saga of the Corleone crime f...
4,667257,Impossible Things,"Matilde is a woman who, after the death of her..."
...,...,...,...
9995,11375,Hollywood Homicide,"Joe Gavilan and his new partner K. C. Calden, ..."
9996,298614,Do Not Disturb,"Michel, who's crazy about jazz, has just found..."
9997,50357,Apollo 18,"Officially, Apollo 17 was the last manned miss..."
9998,97795,Apartment 143,A team of parapsychologists sets out to invest...


In [42]:
m=joblib.load('movie_list.joblib')

In [37]:
joblib.load('similarity.pkl')

array([[1.        , 0.        , 0.0244998 , ..., 0.02178551, 0.        ,
        0.        ],
       [0.        , 1.        , 0.0285831 , ..., 0.02541643, 0.        ,
        0.        ],
       [0.0244998 , 0.0285831 , 1.        , ..., 0.        , 0.        ,
        0.08318903],
       ...,
       [0.02178551, 0.02541643, 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.08318903, ..., 0.        , 0.        ,
        1.        ]])

In [43]:
s=joblib.load('similarity.joblib')

In [44]:
def recommendation(film):
    index=m[m['title']==film].index[0]
    distance=sorted(list(enumerate(s[index])),reverse=True,key=lambda vector:vector[1])
    for i in distance[0:5]:
        print(movies.iloc[i[0]].title)

In [45]:
recommendation('Iron Man')

Iron Man
Clown
Iron Man 2
The Wizards Return: Alex vs. Alex
G.O.R.A.


In [68]:
!pip freeze> requirements.txt

/usr/bin/pip:6: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import load_entry_point


In [79]:
from sklearn.feature_extraction.text import CountVectorizer

# Sample documents
documents = [
    "This is the first document.",
    "This document is the second document.",
    "And this is the third one.",
    "Is this the first document?",
]

# Create an instance of CountVectorizer
vectorizer = CountVectorizer()

# Fit the vectorizer to the documents and transform the documents into a document-term matrix
X = vectorizer.fit_transform(documents)

# Get the feature names (tokens)
feature_names = vectorizer

# Print the feature names
# print(feature_names)

# # Print the document-term matrix
# print(X.toarray())

In [80]:
print(X)

  (0, 8)	1
  (0, 3)	1
  (0, 6)	1
  (0, 2)	1
  (0, 1)	1
  (1, 8)	1
  (1, 3)	1
  (1, 6)	1
  (1, 1)	2
  (1, 5)	1
  (2, 8)	1
  (2, 3)	1
  (2, 6)	1
  (2, 0)	1
  (2, 7)	1
  (2, 4)	1
  (3, 8)	1
  (3, 3)	1
  (3, 6)	1
  (3, 2)	1
  (3, 1)	1


In [82]:
print(X.toarray())

AttributeError: 'list' object has no attribute 'toarray'

In [69]:
print(X)

  (0, 8)	1
  (0, 3)	1
  (0, 6)	1
  (0, 2)	1
  (0, 1)	1
  (1, 8)	1
  (1, 3)	1
  (1, 6)	1
  (1, 1)	2
  (1, 5)	1
  (2, 8)	1
  (2, 3)	1
  (2, 6)	1
  (2, 0)	1
  (2, 7)	1
  (2, 4)	1
  (3, 8)	1
  (3, 3)	1
  (3, 6)	1
  (3, 2)	1
  (3, 1)	1


In [93]:
import numpy as np

# Create a NumPy array of ASCII strings
array = np.array(['This is a string', 'This is another string'])
print(array)
print(type(array))
# Convert the data type of the array to Unicode string
a = array.astype('U')

# Print the array
print(a)
print(type(a))

['This is a string' 'This is another string']
<class 'numpy.ndarray'>
['This is a string' 'This is another string']
<class 'numpy.ndarray'>
